# Model Intercomparison

In [ ]:
import sys
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

sys.path.append('..')

from ideal_geometry import IdealGeometry
from Plume import PlumeModel
from PICO import PicoModel
from PICOP import PicoPlumeModel

%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2

## Experiment 1: melt rated for reference setups `test1` and `test3`

In [ ]:
from ideal_geometry import cases
from real_geometry import glaciers

In [ ]:
# 1D plume model


In [ ]:
models = ['Plume', 'PICO', 'PICOP']
kw = dict(vmin=0, vmax=50, shading='auto', cmap='plasma')

f, ax = plt.subplots(3,len(models)+1, figsize=(12,9), sharey=True, sharex=True)
ax[0,0].set_title('draft')
for i, testcase in enumerate(['test1', 'test2', 'test3']):
#     print('\n', testcase, '\n')
    ds = IdealGeometry(testcase).create()
    ax[i,0].pcolormesh(ds.x/1e3, ds.y/1e3, ds.draft, shading='auto', vmin=-1000, vmax=-500)
    ax[i,0].set_ylabel('y  [km]')
    for j, model in enumerate(models):
        if i==0:  ax[0,j+1].set_title(model)
        if model=='Plume':
            results = PlumeModel(ds).compute_plume()
            ax[i,j+1].pcolormesh(results.x/1e3, results.y/1e3, results.m, **kw)
            melt = results.m.mean().values
        elif model=='PICO':
            _, results = PicoModel(name=testcase).compute_pico()
            ax[i,j+1].pcolormesh(results.x/1e3, results.y/1e3, results.melt, **kw)
            melt = results.melt.mean().values
#             print(f'({results.mk[0].values:.1f} from within PICO)')            
        elif model=='PICOP':
            _, _, results = PicoPlumeModel(name=testcase, n=3).compute_picop()
            ax[i,j+1].pcolormesh(results.x/1e3, results.y/1e3, results.m, **kw)
            melt = results.m.mean().values
        ax[i,j+1].text(.95,.9, f'{melt:.2f} m/yr', transform=ax[i,j+1].transAxes, ha='right', color='w', fontsize=12)
            
for j in range(4):
    ax[-1,j].set_xlabel('x  [km]')


In [ ]:
models = ['Plume', 'PICO', 'PICOP', 'Sheet']
kw = dict(vmin=0, vmax=50, shading='auto', cmap='plasma')

f, ax = plt.subplots(3,len(models)+1, figsize=(15,9), sharey=True, sharex=True)
ax[0,0].set_title('draft')
for i, testcase in enumerate(['test1', 'test2', 'test3']):
    ds = IdealGeometry(testcase).create()
    ax[i,0].pcolormesh(ds.x/1e3, ds.y/1e3, ds.draft, shading='auto', vmin=-1000, vmax=-500)
    ax[i,0].set_ylabel('y  [km]')
    for j, model in enumerate(models):
#         if j<3:  continue
        if i==0:  ax[0,j+1].set_title(model)
        if model=='Plume':
            results = PlumeModel(ds).compute_plume()
            ax[i,j+1].pcolormesh(results.x/1e3, results.y/1e3, results.m, **kw)
            melt = results.m.mean().values
        elif model=='PICO':
            _, results = PicoModel(name=testcase).compute_pico()
            ax[i,j+1].pcolormesh(results.x/1e3, results.y/1e3, results.melt, **kw)
            melt = results.melt.mean().values
            # print(f'({results.mk[0].values:.1f} from within PICO)')            
        elif model=='PICOP':
            _, _, results = PicoPlumeModel(name=testcase, n=3).compute_picop()
            ax[i,j+1].pcolormesh(results.x/1e3, results.y/1e3, results.m, **kw)
            melt = results.m.mean().values
        elif model=='Sheet':
            if i==1: continue
            results = xr.open_dataset(f'../../results/Sheet/Sheet_T0_S34_{testcase}.nc')
            ax[i,j+1].pcolormesh(results.x/1e3, results.y/1e3, results.melt*3600*24*365, **kw)
            melt = results.melt.mean().values*3600*24*365
        ax[i,j+1].text(.95,.9, f'{melt:.2f} m/yr', transform=ax[i,j+1].transAxes, ha='right', color='w', fontsize=12)
            
for j in range(4):
    ax[-1,j].set_xlabel('x  [km]')


## temperature dependence

In [ ]:
# from Erwin's runs
sheet = [[ 2.5686432,   6.18703265, 10.25362111, 14.77641703, 19.75861614, 25.19942571, 31.0948176,  37.43819347, 44.22095555],
         [ 2.94456839,  7.12903415, 11.71694807, 16.71844499, 22.14056505, 27.98734453, 34.25992276, 40.95688662, 48.0747288 ]]

In [ ]:
f, ax = plt.subplots(1,2,figsize=(10,4), sharey=True, constrained_layout=True)
ax[0].set_ylabel('mean melt rate [m/yr]')
Tas = np.arange(-2,2.5,.5)
for i, testcase in enumerate(['test1', 'test3']):
    ref = ax[i].axvspan(-.15,.15, alpha=.1, color='purple', label='reference')
    sm = ax[i].scatter(np.arange(-2,2.1,.5), sheet[i], c='k', label='Sheet')
    lb, = ax[i].plot(Tas,  7*(Tas+2), c='lightgrey', label='7 m/yr/K')
    ub, = ax[i].plot(Tas, 16*(Tas+2), c='darkgrey', label='16 m/yr/K')
    ax[i].set_title(testcase)
    ax[i].set_xlabel(r'ambient temperature [$^\circ$C]')
    for T in Tas:
        handles = []
        for j, model in enumerate(models):
            if model=='Plume':
                c = 'C0'
                pdict={'Ta':T}
                ds = IdealGeometry(name=testcase, pdict=pdict).create()
                results = PlumeModel(ds).compute_plume()
                melt = results.m.mean().values
            elif model=='PICO':
                c = 'C1'
                _, results = PicoModel(name=testcase, Ta=T).compute_pico()
                melt = results.melt.mean().values
            elif model=='PICOP':
                c = 'C2'
                _, _, results = PicoPlumeModel(name=testcase, Ta=T, n=3).compute_picop()
                melt = results.m.mean().values
            s = ax[i].scatter(T, melt, c=c, label=model)
            handles.append(s)
    ax[i].legend(handles=[ref]+handles[:3]+[sm,lb,ub])